In [50]:
from pot_correction import *

In [51]:
DNA_LENGTH = 4000           
READ_lENGTH = 50 
READ_COUNT = 16000 
KMER_LENGTH = 20 

In [52]:
dna = generate_dna(DNA_LENGTH)
reads = [generate_read(dna, READ_lENGTH, lambda base: gauss_unsharp_uncertainty_generator(base, 0.46)) for _ in range(READ_COUNT)]

In [53]:
most_likely_restorer_error_rate(reads)

[0.88, 0.9, 0.9, 0.94, 0.82, 0.86, 0.9, 0.9, 0.88, 0.82, 0.8, 0.8, 0.8, 0.8, 0.9, 0.84, 0.8, 0.76, 0.88, 0.74, 0.84, 0.78, 0.82, 0.78, 0.86, 0.82, 0.82, 0.8, 0.9, 0.86, 0.92, 0.8, 0.86, 0.84, 0.9, 0.84, 0.84, 0.88, 0.8, 0.86, 0.84, 0.92, 0.9, 0.9, 0.84, 0.92, 0.88, 0.8, 0.88, 0.74, 0.9, 0.84, 0.86, 0.78, 0.84, 0.82, 0.86, 0.76, 0.86, 0.88, 0.9, 0.82, 0.84, 0.84, 0.82, 0.86, 0.94, 0.88, 0.9, 0.94, 0.8, 0.86, 0.86, 0.9, 0.84, 0.86, 0.9, 0.84, 0.7, 0.94, 0.9, 0.9, 0.82, 0.88, 0.86, 0.88, 0.92, 0.84, 0.82, 0.84, 0.78, 0.74, 0.86, 0.9, 0.78, 0.82, 0.88, 0.9, 0.88, 0.86, 0.82, 0.86, 0.8, 0.8, 0.88, 0.76, 0.92, 0.76, 0.76, 0.86, 0.86, 0.9, 0.9, 0.86, 0.86, 0.86, 0.8, 0.8, 0.8, 0.92, 0.9, 0.88, 0.84, 0.82, 0.92, 0.82, 0.94, 0.92, 0.84, 0.92, 0.78, 0.86, 0.84, 0.88, 0.9, 0.74, 0.88, 0.84, 0.76, 0.8, 0.86, 0.92, 0.88, 0.78, 0.86, 0.8, 0.84, 0.8, 0.88, 0.94, 0.8, 0.84, 0.9, 0.8, 0.84, 0.8, 0.84, 0.86, 0.88, 0.82, 0.88, 0.82, 0.84, 0.9, 0.82, 0.84, 0.82, 0.82, 0.84, 0.82, 0.74, 0.84, 0.82, 0.86, 0

0.14952750000000214

In [54]:
kmer_hash = KmerHash(KMER_LENGTH, reads)

In [55]:
r = reads[6]
kmer = most_likely_restorer(r.uncertain_text)[:KMER_LENGTH]

In [56]:
adjacent_reads = kmer_hash.get_adjacent_reads(kmer, 3)

In [57]:
len(adjacent_reads)

4

In [58]:
adjacent_reads = adjacent_reads[:20]

In [59]:
most_likely_restorer_error_rate(adjacent_reads)

[0.88, 0.96, 0.9, 0.94]


0.08000000000000007

In [60]:
pot = Pot()
pot.members = set(adjacent_reads)
in_pot_alignment, in_pot_score = get_in_pot_alignment(pot)
restored_text = most_likely_pot_string(in_pot_alignment)
corrected_reads = correct_reads_with_restored_text(in_pot_alignment, restored_text)

In [61]:
most_likely_restorer_error_rate(corrected_reads)

[1.0, 0.98, 0.98, 1.0]


0.010000000000000009